#  <p><center><span style="color:red"> TP 1 : Page Rank Algorithm </span></center></p>

>##       <u>     1) Practising with Python </u> 



### Construction of L and M 

Construction of a list and of it squares 

In [4]:
import numpy as np 

L=np.array([1,2,3])
M=L**2

print(L,M)

[1 2 3] [1 4 9]


### List of even integers 

Function that returns the even integers of a list 

In [5]:
def even(list):
    even=[]
    for i in list:
        if i%2==0:
            even.append(i)
    return even

even(M)

[4]

### List of integers in a file 

Function that reads the integers in a file 

In [11]:
def list_of_file(file):
    f=open(file,"r")
    list=[]
    for line in f:
        u,v= [ int(x) for x in line.split() ]
        list.append(u)
        list.append(v)
    return list 

list_of_file('./graphe')

[1, 2, 2, 3, 3, 4, 4, 1, 1, 5, 5, 6, 6, 1]

## <u> 2) Implementing PageRank in Python </u>

### 1) a) Beta=1 ; eps = 0.1

First, I define two functions that will be useful to create the function PageRank

In [12]:
def distance(r1,r2): #Return the norm1 of the vector r1-r2 (to see the difference between r1 and r2)
    res=0
    n=len(r1)
    for i in range(n):
        res+=abs(r1[i]-r2[i])
    return res

def product(d,v): #Return the product of a sparse matrice represented by a dictionnary d and a vector v 
    n=len(v)
    res=np.zeros(n)
    for i in range(n):
        l=d[i+1]
        for x in l:
            res[x-1]+=(1./len(l))*v[i]
    return res

Here is the implementation of the function PageRank.<br />
It enables to apply the PageRank algorithm to a graph writtend in a file. <br />
We can adjust the Beta and the epsilon parameters. 

In [13]:
def PageRank(file,beta,eps):
    f = open(file,'r')
    d={}
    maxi=0
    for line in f:                                   # creation of the sparse matrix, extracted from the file 
        u,v= [int(x) for x in line.split()]          # and research of the maximum of the graph's indexes maxi 
        if u in d:
            d[u].append(v)
        else:
            d[u]=[v]
        if u>maxi:
            maxi=u
        if v>maxi:
            maxi=v
    r=[]
    for i in range(maxi):
        r.append(1.0)
    r=np.array(r)                                    # creation of the initial vector result r 
    r=r/maxi
    p=(1-beta)*r
    while distance(r,beta*product(d,r)+p)>eps: 
        r=beta*product(d,r)+p                     #iteration until the distance between the old and the new r is superior to eps
    y={}
    n=len(r)
    for i in range(1,n+1):
        y["node "+str(i)]=r[i-1]
    return "There you can see the importance of each node between 0 and 1 : ", y

In [15]:
PageRank('./graphe',1,0.001)

('There you can see the importance of each node between 0 and 1 : ',
 {'node 1': 0.28576151529947913,
  'node 2': 0.14273834228515625,
  'node 3': 0.14288075764973956,
  'node 4': 0.14300028483072916,
  'node 5': 0.14273834228515625,
  'node 6': 0.14288075764973956})

The node 1 is the most important in so far as two nodes target the node 1. 
The others are less important. 

### 1 )b) Beta=0.8 ; esp=0.1

We change the parameter Beta, which represents the jump possibility and we obtain : 

In [16]:
PageRank('./graphe',0.8,0.001)

('There you can see the importance of each node between 0 and 1 : ',
 {'node 1': 0.27144166191245994,
  'node 2': 0.14205073558667264,
  'node 3': 0.14688009681043457,
  'node 4': 0.15069667329332564,
  'node 5': 0.14205073558667264,
  'node 6': 0.14688009681043457})

## 2) Web Graph : applying PageRank to Wikipedia pages 

Now, we have to create the graph ourself,by exploring the html code of each Wikipedia page, and by seeing if they contains links to other pages. 

In [3]:
import re
from os import listdir
from os.path import isfile, join
import html


files = [f for f in listdir("./toyset") if isfile(join("./toyset", f))]

g=[]
filesSet=set(files)
graph=""
for i in range(len(files)):
    f=open("./toyset/"+files[i],encoding="utf8")
    
    #Getting the list of links in current html file
    linkList=re.findall(r'href=[\'"]?([^\'" >]+)',f.read())
    
    #Remove all unwanted links by intersecting with files
    linkList=list(set(linkList) & filesSet)
    for link in linkList:
        v=files.index(link)
        graph+=str(i)+" "+str(v)+","
        
graph=graph[:-1] #Remove the last coma

f = open("wikipedia_graph.txt", "w")
n = f.write(graph)
f.close()

pi=PageRank("wikipedia_graph.txt",0.8,0.001).toarray()[0]
print(pi)
for i in range(len(pi)):
    print(files[i]+" : "+str(pi[i]))



UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 37: invalid start byte

Now, we have our graph, named "graph_wiki". We adapt a little our function PageRank to presents the results differently. 

In [2]:
def PageRank_graph(graph,beta,eps):
    d={}
    maxi=0
    for couple in graph:                                   # creation of the sparse matrix 
        u,v= couple[0],couple[1]
        if u in d:
            d[u].append(v)
        else:
            d[u]=[v]
        if u>maxi:
            maxi=u
        if v>maxi:
            maxi=v
    r=[]
    for i in range(maxi):
        r.append(1.0)
    r=np.array(r)
    r=r/maxi
    p=(1-beta)*r
    while distance(r,beta*product(d,r)+p)>eps: 
        r=beta*product(d,r)+p #iteration until the distance between the old and the new r is superior to eps
    n=len(r)
    y={}
    for i in range(1,n+1):
        y[dico[i]]=100*r[i-1]
    return "There you can see the importance of each page between 0 and 100 : ", y

PageRank_graph(graph_wiki,1,0.1)


NameError: name 'np' is not defined